# Benchmark for vibration of a string

Assumptions made:
- string is treated as perfectly flexible (no stiffness)
- fixed-fixed condition so that reflections are perfectly inverted

Goal: simulate the kink resulting from pluckign the string

Constituitive equation: $\frac{\partial^2 y}{\partial t^2} = c^2 \frac{\partial^2 y}{\partial x^2}$

In [1]:
using CombinatorialSpaces
using CombinatorialSpaces.DiscreteExteriorCalculus: inv_hodge_star
using DifferentialEquations
using CairoMakie

# dual subdivision 
function dual(s::EmbeddedDeltaSet2D{O, P}) where {O, P}
  sd = EmbeddedDeltaDualComplex2D{O, eltype(P), P}(s)
  subdivide_duals!(sd, Circumcenter())
  sd
end

LoadError: UndefVarError: DiscreteExteriorCalculus not defined

## Define mesh

In [ ]:
# Define mesh
s = EmbeddedDeltaSet2D("meshes/pipe_fine.stl")
sd = dual(s)

figure, axis, ob = wireframe(s)
axis.aspect = AxisAspect(10/1)
figure